In [ ]:
from csv import DictReader

departures = {}
with open('../../data/navigo_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if \
                row['destination_fr'] == 'Marseille' \
            and row['indate_fixed'].split('-')[0] == '1789' \
            and row['departure_fr'] != '' \
            :
            departure = row['departure_fr']
            if departure not in departures:
                departures[departure] = {
                    "latitude": float(row["departure_latitude"].strip() or 0),
                    "longitude": float(row["departure_longitude"].strip() or 0),
                    "count": 0
                }
            departures[departure]['count'] += 1

In [ ]:
from ipyleaflet import Map, Marker, Popup, basemaps, basemap_to_tiles, CircleMarker
from ipywidgets import HTML

# this function allows to map a value from a domain of min-max to another
def map_value(value, domain_min, domain_max, range_min, range_max):
    left_span = domain_max - domain_min
    right_span = range_max - range_min

    # Convert the left range into a 0-1 range (float)
    scaled = float(value - domain_min) / float(left_span)

    # Convert the 0-1 range into a value in the right range.
    return range_min + (scaled * right_span)

# doc : https://ipyleaflet.readthedocs.io/en/latest/api_reference/circle_marker.html

In [ ]:
# compute min and max for radiuses
ports = [port for key,port in departures.items()]
max_count = max([port["count"] for port in ports])
min_count = min([port["count"] for port in ports])
MIN_RADIUS = 1
MAX_RADIUS = 20

# setup map
center = (47, -1) # France
portic_map = Map(center=center,zoom=5)

# plot data
for port, port_data in departures.items():
    circle_marker = CircleMarker()
    circle_marker.location = (port_data["latitude"], port_data["longitude"])
    circle_marker.radius = int(map_value(port_data["count"], min_count, max_count, MIN_RADIUS, MAX_RADIUS))
    circle_marker.color = "red"
    circle_marker.fill_color = "red"
    portic_map.add_layer(circle_marker)
          
display(portic_map)

In [ ]:
from keplergl import KeplerGl
import pandas as pd

# setting kepler config
config = {
    'version': 'v1',
    'config': {
        # centering the map on the region
        'mapState': {
            'latitude': 45.6876849,
            'longitude': -1.15,
            'zoom': 5.2
        }
    }
}

df = pd.DataFrame([{**values, "size": values["count"]} for values in departures.values()])

points_map = KeplerGl(config=config, data={'data_1': df})
points_map